# CSX46 Homework Assignment 2: Network Motif Analysis

Load in the *E. coli* operon-operon network (gene regulatory network) data in edge-list format; display the first six lines of data as a sanity check. Use `pandas.read_csv` and `pandas.DataFrame.head`.

In [2]:
import pandas
gene_df = pandas.read_csv("shared/ecolitfnet.txt",
                           sep="\t",
                           names=["source","target"]).drop_duplicates()
print(gene_df.head())

  source target
0    ADA   AIDB
1   ADIY   GADX
2   ALLR   ALLS
3   ARCA   BETI
4   ARCA   DPIA


Make an igraph directed graph from the network; print a graph summary as a sanity check.  You will need to use `igraph.Graph.TupleList` and `igraph.Graph.summary`.

In [3]:
import igraph
from igraph import Graph
gene_graph = Graph.TupleList(gene_df[["source","target"]].values.tolist(),
                                 directed=True)
gene_graph.summary()

'IGRAPH DN-- 133 261 -- \n+ attr: name (v)'

## Part I: Three-vertex motif analysis

Obtain the counts of all three-vertex motifs in the graph. You will need to use the `motifs_randesu` method in `igraph.Graph`.

*Question 1: Which one of these motifs is most frequent in the E. coli regulatory network?*

In [4]:
import math
gene_motifs_3 = gene_graph.motifs_randesu(size = 3)
for index, value in enumerate(gene_motifs_3):
    if math.isnan(value):
        gene_motifs_3[index] = 0
gene_motifs_sorted = sorted(enumerate(gene_motifs_3), 
                          key=lambda x:x[1],
                          reverse=True)
print(gene_motifs_sorted[0])

(6, 1676)


use motifs_randesu find different motifs type and is it's nan change it to 0 then sorted to find most frequent. we find the most is class 6 motif

HINT: The most frequent motif has 1,676 occurrences in the network.

*Question 2: Which one of these motifs has a count of 47 in the regulatory network? I’ll refer to this specific motif as the “mystery motif”, or MM.*

In [5]:
MM = {index: value for index, value in enumerate(gene_motifs_3) if value == 47}
print(MM)

{7: 47}


find the motifs number is 47, it's class 7 type of motif

Get motif counts for 10,000 different randomizations of the network.  You will need to use a `for` loop, `igraph.Graph.copy`, `igraph.Graph.rewire`, and `append`.  You can specify n=1000 for the rewiring.

In [7]:
gene_MM_motif = []
gene_graph_copy = Graph.copy(gene_graph)
for i in range(10000):
    gene_graph_copy.rewire(n = 1000)
    gene_MM_motif.append(gene_graph_copy.motifs_randesu(size = 3)[7])
print(gene_MM_motif)

[51, 59, 56, 56, 55, 69, 69, 76, 59, 41, 50, 62, 80, 63, 76, 53, 42, 60, 45, 55, 61, 65, 55, 61, 59, 64, 53, 54, 60, 58, 53, 63, 50, 66, 52, 60, 42, 73, 49, 60, 49, 52, 58, 40, 63, 50, 49, 59, 66, 67, 60, 48, 76, 64, 55, 40, 76, 63, 72, 52, 49, 47, 42, 64, 48, 69, 43, 66, 61, 51, 52, 63, 65, 74, 42, 54, 61, 55, 52, 55, 73, 58, 45, 56, 64, 72, 68, 48, 51, 59, 48, 46, 65, 71, 58, 59, 63, 54, 52, 69, 62, 46, 77, 56, 47, 68, 69, 61, 53, 49, 68, 66, 42, 46, 53, 67, 41, 56, 57, 60, 76, 54, 47, 72, 81, 49, 64, 75, 50, 68, 58, 59, 68, 66, 49, 53, 54, 48, 58, 59, 60, 73, 62, 45, 64, 57, 73, 53, 54, 62, 55, 48, 55, 63, 44, 59, 55, 68, 70, 78, 48, 72, 46, 55, 51, 59, 51, 53, 49, 74, 47, 62, 62, 75, 64, 49, 55, 67, 53, 56, 46, 52, 48, 53, 59, 53, 40, 46, 51, 50, 60, 67, 47, 51, 60, 58, 66, 40, 63, 57, 60, 60, 66, 74, 65, 59, 56, 68, 42, 55, 46, 69, 65, 48, 59, 38, 65, 62, 53, 59, 45, 53, 58, 47, 48, 67, 68, 52, 65, 59, 52, 58, 42, 56, 61, 39, 56, 59, 56, 70, 57, 56, 60, 67, 57, 47, 55, 44, 60, 49,

append all number of MM in different random network use rewire, then we can find all different

For each motif type, compute the mean and standard deviation of the counts of appearances of the motif in each of the 10,000 randomly rewired graphs. You will need to use the `numpy.mean` and `numpy.std` functions.

In [11]:
import numpy
gene_MM_np = numpy.array([item for item in gene_MM_motif])
gene_MM_mean = numpy.mean(gene_MM_np)
gene_MM_std = numpy.std(gene_MM_np)

use numpy.array to build a array then use mean and std to cauculate the mean and standard deviation of MM

For each motif type, obtain the *Z* score for the number of appearances of the motif in the real network, vs. in the randomly rewired networks; store this in object `motif_counts_zscores`.

In [17]:
motif_counts_zscores = (47 - gene_MM_mean) / gene_MM_std

z score = (Q_real - Q_random_mean) / std

*Question:  What are the mean and standard deviation of the MM counts for the 10,000 random networks?*

In [15]:
print("mean:", gene_MM_mean)
print("standard deviation:",gene_MM_std)

mean: 57.3979
standard deviation: 9.231986546242362


*Question 3: What is the Z-score for enrichment of the MM count for the real network vs. the ensemble of random networks? Does this z-score correspond to a statistically significant positive enrichment?*

In [18]:
print("Z-score:", motif_counts_zscores)

Z-score: -1.1262906361396499


NO, because the Z-score is negative

*Question 4: What is the ratio of the MM count for the real network to the average MM count for the random networks?*

In [20]:
ratio = 47 / gene_MM_mean
print(ratio)

0.81884528876492


*Question 5: How does ratio compare to the same ratio for the data in Table 1 in Shen-Orr et al., Nature Genetics, 2002?*

answer: from my result of ratio is much smaller than ratio from Shen-Orr et al.

*Question 6: Given the modest ratio of the MM frequency in the real network vs. randomly shuffled network, should we entertain the possibility that the high frequency of MMs in the real network could be a consequence of the degree distribution rather than evolution specifically “favoring” FFLs as a building block for gene regulatory networks?*

answer: I think it's ture because we build random network with the same degree so when we have the modest ratio it means the frequency is related to the degree of vertices. if only real network has high frequency it means evolution help FFLs to be a high frequent motif.

## Part II:  Four-vertex motif analysis

*Question 1: What is the count of this four-vertex motif in the E. coli gene regulatory network?*

Figure out the isomorphism class of the network shown in the assignment PDF.  You'll want to build an edge-list tuple-list and then an igraph `Graph` object using `Graph.TupleList` (make sure to specify a directed graph!), and then use the `isoclass` method.

In [44]:
data = {"source":["H", "H", "GADW", "GADW"],
        "target":["GADE","GADX","GADE","GADX"]}
iso_df = pandas.DataFrame(data)
iso_graph = Graph.TupleList(iso_df[["source","target"]].values.tolist(),
                                 directed=True)
iso_graph.isoclass()

19

define a isomorphism graph of this specific class, then use isoclass to find that it belong to class 19

Compute the number of appearances of this four-vertex motif in the real network, using `motifs` with `size=4`.

In [50]:
gene_motifs_4_all = gene_graph.motifs_randesu(size = 4)
print(gene_motifs_4_all[19])

297


then we use this class number to find the number of appearances of this motif

*Question 2: What are the average (and standard deviation) of the counts of this motif in 10,000 randomly shuffled networks?*

Get the mean and standard deviation for the count of four-vertex motifs in 10,000 randomly rewired graphs.  You will need to use a `for` loop, `igraph.Graph.copy`, `igraph.graph.rewire`, and `append`.  You can specify n=1000 for the rewiring.

In [45]:
gene_motif_4 = []
gene_graph_copy = Graph.copy(gene_graph)
for i in range(10000):
    gene_graph_copy.rewire(n = 1000)
    gene_motif_4.append(gene_graph_copy.motifs_randesu(size = 4)[19])
print(gene_motif_4)

[228, 170, 166, 146, 148, 197, 160, 198, 227, 207, 168, 149, 182, 146, 236, 243, 86, 130, 190, 164, 199, 190, 152, 190, 185, 157, 258, 179, 196, 241, 138, 164, 156, 165, 201, 182, 136, 167, 204, 223, 163, 170, 127, 135, 201, 213, 166, 301, 145, 187, 185, 190, 190, 159, 150, 204, 117, 139, 153, 130, 200, 155, 168, 212, 115, 198, 191, 160, 257, 135, 196, 97, 183, 243, 210, 173, 237, 258, 229, 150, 121, 156, 131, 131, 177, 139, 212, 154, 185, 104, 149, 208, 120, 188, 186, 190, 126, 236, 137, 120, 183, 254, 159, 249, 177, 172, 241, 271, 178, 242, 140, 139, 169, 149, 238, 160, 164, 124, 184, 141, 205, 160, 143, 238, 180, 222, 228, 168, 169, 140, 171, 152, 145, 134, 197, 130, 150, 257, 202, 228, 201, 279, 123, 146, 270, 168, 110, 207, 122, 248, 200, 241, 190, 204, 219, 164, 211, 148, 163, 174, 165, 189, 228, 147, 190, 153, 183, 195, 191, 211, 196, 183, 117, 179, 169, 185, 257, 149, 180, 138, 151, 148, 233, 183, 192, 206, 134, 134, 177, 198, 200, 234, 141, 98, 125, 98, 247, 154, 211, 171, 209

for 10000 loop append all class 19 motifs into one list

Print out the mean number of 4-vertex DOR motifs for the random networks, the standard deviation, and the Z score for the DOR motif count in the real graph vs. the random graphs. You'll need to use `numpy.array`, `numpy.mean`, and `numpy.std`.

In [61]:
gene_motif_4_np = numpy.array([item for item in gene_motif_4])
gene_motif_4_mean = numpy.mean(gene_motif_4_np)
gene_motif_4_std = numpy.std(gene_motif_4_np)
print("mean:", gene_motif_4_mean)
print("standard deviation:",gene_motif_4_std)

mean: 178.3196
standard deviation: 38.82200221317803


use the list to find the mean and standard deviation of this motif

*Question 3: What is the Z-score for enrichment of this four-vertex motif?  Is it significant?*

In [62]:
motif_4_zscore = (gene_motifs_4_all[19] - gene_motif_4_mean) / gene_motif_4_std
print("Z-score:", motif_4_zscore)

Z-score: 3.0570396485041216


calculate the Z-score of this motif. it's significant positive enrichment

You may find `scipy.stats.norm.cdf` useful

In [63]:
from scipy.stats import norm
norm.cdf(gene_motifs_4_all[19],gene_motif_4_mean, gene_motif_4_std)

0.9988823264681181

*Question 4: What is the ratio of the count in the real network to the average count for the random networks?  How does this ratio compare to the ratio that you obtain from Table 1 from Shen-Orr et al.? Are they consistent? Does this suggest that Shen-Orr’s actual network randomization procedure is possibly not consistent with their description in the Methods section of their paper, i.e., that it may have had some kind of error?*

In [60]:
ratio_4 = gene_motifs_4_all[19] / gene_motif_4_mean
print(ratio_4)

1.6655488235729554


It's smaller than the ratio from Shen-Orr et al. I think it's more frequency appeared in random network. they said that X→Y1, X2→Y2 is  replaced by X1→Y2, X2→Y1 to randomize the network.